In [2]:
from IPython.display import HTML
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import animation

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
The raw code for this Jupyter notebook is by default hidden for easier reading.
To toggle on/off the raw code, click <a href="javascript:code_toggle()">here</a>.
<style>
.output_png {
    display: table-cell;
    text-align: center;
    vertical-align: middle;
}
</style>
''')

![title](figs/COMPGEOP_TITLE.jpg)

### <h1><center>Module 10: Numerical Solutions to Hyperbolic PDEs</center></h1>

This module focuses on generating numerical solutions of **hyperbolic** partial differential equations (PDEs).  Recall that these types of second-order PDEs

$$Au_{xx}+2Bu_{xy}+Cu_{yy}+Du_{x}+Eu_{y}+Fu+G=0 \tag{1}$$

are defined by the condition $B^2-AC > 0$ (where $y$ is considered the $t$ variable and thus $A=1$, $C=-\frac{1}{c}$ and all others are equal 0).

Three interesting examples (with increasing complexity) are the following:

   * **Acoustic wave equation (Homogeneous medium)**: The acoustic wave equation (AWE) is normally used to model the pressure disturbance $\phi=\phi(x,y,z,t)$ generated by an acoustic wave propagation through a fluid medium defined by constant velocity $v=\sqrt{\frac{K}{\rho}}$ where $K$ is the bulk compressibility and $\rho$ is the density.
   
$$\frac{\partial^2 \phi}{\partial t^2} - v^2\nabla^2 \phi = F(x,y,z,t) \tag{2a}$$ 

where $F(x,y,z,t)$ is an external force term. Because this is a second-order temporal partial derivative, to uniquely define the wavefield solution one needs to define two **initial conditions** $\phi(x,y,z,t=0)=f(x,y,z)$ and $\frac{\partial \phi}{\partial t}(x,y,z,t=0)=\phi_t=g(x,y,z)$. (Note that the subscript indicates which partial derivative has been applied.) One also has to prescribe the behavior of the solution on the domain boundary $D$ (e.g., $\left. \phi\right|_{\partial D}=B(x,y,z)$). Note that the acoustic wave equation is commonly used in exploration seismology to approximate compressional (i.e., P-wave) propagation through complex heterogeneous media.
   
   * **Acoustic wave equation (variable density)**: This is like the example above, except now that the density is assumed to spatially vary:

$$\frac{\partial^2 \phi}{\partial t^2}  -  K \nabla \cdot \left(\frac{1}{\rho} \nabla \phi  \right) = F(x,y,z,t). \tag{3b}$$ 
   
This equation is commonly used in computational ocean acoustics because it allows one to introduce variations in density caused by temperature and salinity concentration among others.  This wave equation requires the same type of initial and boundary conditions as the homogenous equation above.


   * **Advection**:  We have actually looked at this hyperbolic equation in Module 7 of the course. One can recognize this to be the case if we **factorize** the 1D AWE given above in the following way
   
$$\left[\frac{\partial^2 }{\partial t^2} - v^2\frac{\partial}{\partial x^2} \right] \phi = 
\left(\frac{\partial }{\partial t}-v \frac{\partial}{\partial x} \right)\left( \frac{\partial }{\partial t}+v \frac{\partial}{\partial x}\right) \phi = F(x,t). \tag{3c}$$ 
 
Here we see that we have the produce of the left- and right-going advection equations. This was recognized in 1747 by French scientist [d'Alembert](https://en.wikipedia.org/wiki/D%27Alembert%27s_formula).  Note that one sometimes see the **d'Alembert operator** $\Box^2$ in various problems in mathematical physics, which is given by:

$$\Box^2 \phi = \left[\frac{\partial^2 }{\partial t^2} - v^2\frac{\partial}{\partial x^2} \right] \phi\tag{3d}$$


In the section below, we will look at generating solutions $\phi$ of the 1D and 2D wave equation throughout a computational domain $D$.  We will also examine different boundary conditions (BCs) on boundary of D (defined by shortform $\delta \Omega$), namely the Dirchelet BC

$$ \phi(\mathbf {x}) = f(\mathbf {x}), \quad \forall \mathbf {x}\in \delta \Omega \tag{4a}$$

and the Neumann BC

$${\frac {\partial \phi}{\partial \mathbf {n} }}(\mathbf {x} )=f(\mathbf {x} )\quad \forall \mathbf {x} \in \partial \Omega \tag{4b}$$

where ${\bf n}$ is a normal vector defined at the grid boundary that points inward.

## 1D Acoustic wave equation

Let's first look at the problem of an acoustic wave propagating on a homogeneous taut string of length $L$ with fixed ends. In this case, we can assume that there is no external forcing function $F(x,t)$ and the string starts from some combination of an initial displacement disturbance $\phi(x,t=0) = f(x)$ and an initial velocity $\phi_t(x,t=0) = g(x)$. Thus, the PDE we are solving is

$$\frac{\partial^2 \phi}{\partial t^2} = v^2\frac{\partial^2 \phi}{\partial x^2}. \tag{5}$$ 

Let's now look at an **explicit** discretization method to solve equation 5 (as opposed to the **implicit** ones like Crank-Nicolson approach we studied in Module 9).  Let's start by using a standard $\mathcal{O}(\Delta x^2,\Delta t^2)$ discretization.

$$\frac{\phi^{n+1}_i - 2\phi^{n}_i +\phi^{n-1}_i}{\Delta t^2} = 
v^2 \left( \frac{\phi^{n}_{i+i} - 2\phi^{n}_i +\phi^{n}_{i-1}}{\Delta x^2} \right) \tag{6}$$

Let's now multiply through by $\Delta t^2$ and use the Courant number $C^2 = \frac{v^2\Delta t^2}{\Delta x^2}$ from the previous modules to write

$$\phi^{n+1}_i - 2\phi^{n}_i +\phi^{n-1}_i = C^2 \left( \phi^{n}_{i+i} - 2\phi^{n}_i +\phi^{n}_{i-1}  \right). \tag{7}$$

Rearranging terms to isolate the unknown quantity at time step $n+1$ on the left-hand side and keep known quantities on the right-hand side leads to:

$$\phi^{n+1}_i =  C^2 \phi^{n}_{i+i} + 2(1-C^2) \phi^{n}_i +C^2 \phi^{n}_{i-1} -\phi^{n-1}_i . \tag{8}$$

We can also explore what happens at the left and right boundaries from equation 8.  First, at the left hand boundary (i.e., $i=0$) we can write

$$\phi^{n+1}_0 =  C^2 \phi^{n}_{1} + 2(1-C^2) \phi^{n}_0 +C^2 \phi^{n}_{-1} -\phi^{n-1}_0. \tag{9}$$

Note that there is a $ \phi^{n}_{-1}$ term in this equation, which represents a point out of the grid! Let's assume that we can set $ \phi^{n}_{-1}=0$, which is kind of like using a **ghost point** which isn't formally defined, but we conjured it up for numerical expediency.  Similarly, on the right-hand side we will have to do the same at ghost point $ \phi^{n}_{N+1}=0$.

Let's first build a 1D AWE solver:

In [7]:
def awe_explicit_solver_homogeneous(Uo,Um,dx,dt,v,LB,RB):
    '''Set up second-order solver of the acoustic wave equation
    usage: U = awe_explicit_solver_homogeneous(Uo,Um,dx,dt,v,LB,RB)
    input: 
        Uo: Acoustic pressure vector (nx)at time step n
        Um: Acoustic pressure vector (nx) at time step n-1
        dx: Spatial sampling
        dt: Temporal sampling
        v : Homogeneous propagation velocity 
        LB: Left  boundary (Homogeneous Dirchelet)
        RB: Right boundary (Homogeneous Dirchelet)
    output:
        Up: Acoustic pressure vector (nx) at time n+1
    dependencies:
        None
    written by Jeff Shragge, jshragge@mines.edu, 10/2019
    '''        
    ## . . Get dimensions
    nx = len(Uo)
    
    ## . . Courant Number number
    CX2 = (v*dt/dx*dx)**2
    
    ## . . Update solution 
    ## . . Note that I am overwriting Um because this wavefield
    ## . . is not needed after this calculation
    Um[1:nx-1] =     CX2 *Uo[2:nx  ]+ \
                2*(1-CX2)*Uo[1:nx-1]+ \
                     CX2 *Uo[0:nx-2]- \
                          Um[1:nx-1]
            
    ## . . Enforce boundary conditions
    U[0   ] = LB
    U[nx-1] = RB
    
    return Um ## . . Return wavefield at time step n+1

Let's now build up a scenario that we are trying to model.  Let's assume that the length is $L=1$m and the velocity is $v=1$ m/s.